## Calculando Containment


Neste notebook, iremos implementar uma funação containment. Tal função irá comparar dois textose analisar a similaridade dos mesmos com relação aos seus n-gramas de interseção.


## Contar N-gram counts

In [3]:
import numpy as np
import sklearn


In [4]:
from sklearn.feature_extraction.text import CountVectorizer

a_text = "Suponha que essa seja uma resposta de uma questão"
s_text = "Suponha que essa seja a respostade de um aluno"

# set n
n = 1

# instantiate an ngram counter
counts = CountVectorizer(analyzer='word', ngram_range=(n,n))

# create a dictionary of n-grams by calling `.fit`
vocab2int = counts.fit([a_text, s_text]).vocabulary_

# print dictionary of words:index
print(vocab2int)

{'suponha': 8, 'que': 3, 'essa': 2, 'seja': 7, 'uma': 10, 'resposta': 5, 'de': 1, 'questão': 4, 'respostade': 6, 'um': 9, 'aluno': 0}
